---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one():
    
    email_network = nx.read_edgelist('email_network.txt', data=[('time', int)], create_using=nx.MultiDiGraph())
    
    return email_network

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [3]:
def answer_two():
        
    graph = answer_one()
    employees_count = len(graph.nodes())
    emails_count = len(graph.edges())
    
    return (employees_count, emails_count)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [4]:
def answer_three():
    
    graph = answer_one()
    
    # part1 is True if this directed graph is strongly connected, False otherwise
    part1 = nx.is_strongly_connected(graph)
    
    # part2 is True if this directed graph is weakly connected, False otherwise
    part2 = nx.is_weakly_connected(graph)
    
    return (part1, part2)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [5]:
def answer_four():
        
    graph = answer_one()

    largest_weakly_connected_component = list(nx.weakly_connected_components(graph))[0]
    node_count_largest_weakly_connected_component = len(list(nx.weakly_connected_components(graph))[0])
    
    return node_count_largest_weakly_connected_component

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [6]:
def answer_five():
        
    graph = answer_one()

    sorted_strongly_connected_components = sorted(list(nx.strongly_connected_components(graph)), key = lambda x: len(x))
    largest_strongly_connected_component = sorted_strongly_connected_components[-1]
    node_count_largest_strongly_connected_component = len(largest_strongly_connected_component)

    return node_count_largest_strongly_connected_component

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [7]:
def answer_six():
        
    graph = answer_one()

    G_sc = max(nx.strongly_connected_component_subgraphs(graph), key=len)
    
    return G_sc

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [8]:
def answer_seven():
        
    G = answer_six()
    average_distance = nx.average_shortest_path_length(G) # Average distance between every pair of nodes in G
    
    return average_distance

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [9]:
def answer_eight():
        
    G = answer_six()
    diameter = nx.diameter(G) # Diameter of G. I.e. The maximum distance between any pair of nodes in G
    
    return diameter

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [25]:
def answer_nine():
       
    G = answer_six()
    diameter = answer_eight()
    
    periphery = set([x for x in nx.eccentricity(G) if nx.eccentricity(G)[x] == diameter])

    return periphery

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [26]:
def answer_ten():
        
    G = answer_six()
    radius = nx.radius(G)
    
    center = set([x for x in nx.eccentricity(G) if nx.eccentricity(G)[x] == radius])
    
    return center

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [62]:
def answer_eleven():
        
    # I.e. Find the node in the periphery of G_sc, which is most connected to other nodes of G_sc. Find the number of nodes 
    # connected to this node. 

    G = answer_six()
    diameter = answer_eight()
    periphery = answer_nine()

    periphery_nodes_distances = [nx.shortest_path_length(G,x) for x in periphery]
    connected_nodes_diameter_distance_apart = [[node for node in x if x[node] == diameter] for x in periphery_nodes_distances]
    num_connected_nodes_diameter_distance_apart = [len(x) for x in connected_nodes_diameter_distance_apart]

    periphery_nodes_and_count_of_nodes_diameter_distance_apart = list(zip(periphery, 
                                                                          num_connected_nodes_diameter_distance_apart))
    periphery_node_with_most_nodes_diameter_distance_apart = sorted(periphery_nodes_and_count_of_nodes_diameter_distance_apart,
                                                                    key = lambda x: x[1])[-1]
    
    return periphery_node_with_most_nodes_diameter_distance_apart

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [113]:
def answer_twelve():
        
    G = answer_six()
    center = list(answer_ten())
    target_node = answer_eleven()[0]
    
    # Use nx.minimum_node_cut which explicitly gives the list of node set which has minimal size for disconnecting, where as
    # nx.node_connectivity gives an approximation of the number of the node set which has minimal size for disconnecting. 
    smallest_num_nodes_disconnect = min([len(nx.minimum_node_cut(G, x, target_node)) for x in center])
            
    return smallest_num_nodes_disconnect

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [115]:
def answer_thirteen():
        
    G = answer_six()
    G_un = nx.Graph(G.to_undirected())
    
    return G_un

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [121]:
def answer_fourteen():
        
    G = answer_thirteen()
    transitivity = nx.transitivity(nx.Graph(G))
    avg_clustering = nx.average_clustering(nx.Graph(G))

    return (transitivity, avg_clustering)